In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import os
import struct
import matplotlib.pyplot as plt

# import cv2

import dicom
# import SimpleITK as sitk 

from swat import *

Connect to the CAS server. 
My \_authinfo file is located in my home directory: ~/\_authinfo and contains as its first line "default user europe\\sbxjen password ...".
Make sure to put the sample images on the CAS server.

In [2]:
conn = CAS('rdcgrd104.unx.sas.com', '53653', authinfo='../../_authinfo')
conn.setsessopt(caslib='CASUSER(sbxjen)')

NOTE: 'CASUSER(sbxjen)' is now the active caslib.


<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.00872s</span> &#183; <span class="cas-user">user 0.007s</span> &#183; <span class="cas-memory">mem 0.277MB</span></small></p>

Load the action sets.

In [4]:
conn.loadactionset(actionset='Image')
conn.loadactionset(actionset='BioMedImage')
conn.loadactionset(actionset='DeepLearn')

NOTE: Added action set 'Image'.
NOTE: Added action set 'BioMedImage'.
NOTE: Added action set 'DeepLearn'.


<div class="cas-results-key"><b>&#167; actionset</b></div>
<div class="cas-results-body">
<div>DeepLearn</div>
</div>
<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.000123s</span> &#183; <span class="cas-memory">mem 0.0282MB</span></small></p>

Read the first patient image from sample_images.7z from https://www.kaggle.com/c/data-science-bowl-2017/data. firstPatient is a swat.cas.results.CASResults.

In [129]:
# Each folder in /u/sbxjen/sample_images/ represents one patient scan = one (3D) patient image.
firstPatientName = '00cba091fa4ad62cc3200a657aeb957e'
firstPatient = conn.image.loadImages(casOut={'caslib':'CASUSER(sbxjen)', 'name':firstPatientName, 'replace':True},
                      addColumns={'Width', 'Height', 'Depth', 'channelType', 'channelCount', 'position', 'orientation', 'spacing'},
                      decompress=False, path='/u/sbxjen/sample_images/' + firstPatientName)
# Each table contains N records, where N is equal to the number of slices for the patient.
conn.table.recordcount(table={'caslib':'CASUSER(sbxjen)', 'name':'00cba091fa4ad62cc3200a657aeb957e'})

NOTE: Loaded 134 images from /u/sbxjen/sample_images/00cba091fa4ad62cc3200a657aeb957e into Cloud Analytic Services table 00cba091fa4ad62cc3200a657aeb957e.


,N
0,134


Read the labels. labels is a swat.cas.table.CASTable. It supports much of the Pandas pandas.DataFrame API.

In [ ]:
labels = conn.load_path('stage1_labels.csv', caslib='casuser')
labels.head()

Below is a 3D image. To be continued ...

In [29]:
conn.image.loadImages(casOut={'caslib':'CASUSER(sbxjen)', 'name':'medical', 'replace':True}, 
                      addColumns={'Width', 'Height', 'Depth', 'channelType', 'channelCount', 'position', 'orientation', 'spacing'},
                      decode=True, path='/u/fivada/Playpens/PlayITK/SampleImages/vhfAnkle.nii')

NOTE: Loaded 1 image from /u/fivada/Playpens/PlayITK/SampleImages/vhfAnkle.nii into Cloud Analytic Services table medical.


<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.518s</span> &#183; <span class="cas-user">user 0.174s</span> &#183; <span class="cas-sys">sys 0.334s</span> &#183; <span class="cas-memory">mem 228MB</span></small></p>

Fetch the first patient image array. fetchedRows is a swat.dataframe.SASDataFrame. firstPatientImage is a 3D (height x length x breadth) np array. I think the intercept has already been added.

Add slice thickness to downsample, as in https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial: def resample.

In [99]:
def sortAndAddThickness(image):
    """
    Update patient image with the thickness of each slice.
    :param      image: the slices making up one image in one CASTable
    :return:    image, sorted by slices["_positionPatient_"] = double(slices["_position_"]) and updated with slices["_thickness_"]
    """
    dim = image['_dimension_'][0]
    image['_positionPatient_'] = pd.Series(np.array([struct.unpack('='+str(dim)+'d', pos[0:dim*8])[2] for pos in image['_position_']], dtype=np.float64))
    image = image.sort_values(by='_positionPatient_')
    image = image.reset_index(drop=True)
    st = np.abs(image["_positionPatient_"].iloc[0] - image['_positionPatient_'].iloc[1])
    print(st)
    image['_thickness_'] = pd.Series(st * np.ones_like(image['_positionPatient_'], dtype=np.int64))
    return image

In [113]:
def getImageArray(images, dimensions, resolutions, imageFormats):
    """
    Get image as 3D image. 
    :param images: 
    :param dimensions:
    :param resolutions:
    :param imageFormats:
    :return: 
    """
    for n in range(images.shape[0]):
        res = np.array(struct.unpack('='+str( dimensions[n])+'q', resolutions[n][0:dimensions[n]*8]), dtype=np.int64)
        res = res[::-1]
        fmt = imageFormats[n]
        nCells = np.prod(res)
        if (fmt==37):
            slice = np.array(struct.unpack('='+str(nCells)+'i', images[n][0:4*nCells]), dtype=np.int32)
            slice = np.reshape(slice, res)
        elif (fmt==35):
            slice = np.array(struct.unpack('='+str(nCells)+'h', images[n][0:2*nCells]), dtype=np.int16)
            slice = np.reshape(slice, res)
        # else:
        #     image = np.array(bytearray(images[n]))
        #     image = np.reshape(image, (res[0], res[1], 3))
        #     image = reverse(image, 2)
        if (n==0):
            imageArray = slice
        else:
            # print(imageArray.shape, image.shape)
            imageArray = np.concatenate([imageArray, slice], axis=0)
            # print('here')
    imageArray[imageArray==-3024] = -1024
    return imageArray

In [114]:
fetchedRows = conn.fetch(table={'caslib':'CASUSER(sbxjen)', 'name':firstPatientName}, sastypes=False, to=300)['Fetch']
fetchedRows = sortAndAddThickness(fetchedRows)
firstPatientImage = getImageArray(fetchedRows['_image_'], fetchedRows['_dimension_'], fetchedRows['_resolution_'], fetchedRows['_imageFormat_'])

2.5


Note that \_spacing\_ is not a valid alternative.

In [115]:
np.array([struct.unpack('='+str(3)+'d', pos[0:3*8])[2] for pos in fetchedRows["_spacing_"]], dtype=np.float64)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.])

Show one of the first patient's slices and show the associated histogram of pixel values.

In [ ]:
slice = firstPatientImage[80]
df = pd.DataFrame(slice)
print(df.head())

plt.gray()
plt.imshow(slice, vmin=0, vmax=1000)
plt.show()

plt.hist(slice.flatten(), bins=80)
plt.show()

In [79]:
conn.image.loadimages(
    casout={'caslib':'CASUSER(sbxjen)', 'name':'train', 'replace':True},
    path='/u/mukabu/mnist_data/mnist_images/train'
)

NOTE: Loaded 60000 images from /u/mukabu/mnist_data/mnist_images/train into Cloud Analytic Services table train.


<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 307s</span> &#183; <span class="cas-user">user 2.01s</span> &#183; <span class="cas-sys">sys 6.59s</span> &#183; <span class="cas-memory">mem 18.1MB</span></small></p>

The above is nice for visualisation purposes, but we should be able to update the thickness at the server-side.

Next steps: 
(1) add thickness as a column, e.g., by \_id\_, 
(2) downsample in all dimensions (RESIZE for 2D, RESCALE?) [1], 
(3) segmentation? (Finding and drawing contours?, BioMedImage.buildSurface), 
(4) colour normalisation and zero centering (NORMALIZE, ADD\_CONSTANT -0.25),
(5) adding some noise (Andrew?), 
(6) turn the thing into a 3D image (???) and 
(7) join with the labels. 
Then the data are ready for a CNN.
[1] Ideally, we would take thickness and PixelSpacing (not available!) into account.

Add thickness as a column on the server-side.

RESIZE
conn.image.processImages does not work -- independent of decompress = True / False.

RESIZE
conn.image.processImages does not work -- independent of decompress = True / False.

In [139]:
IMG_SIZE = 100
conn.image.processImages(casout={'caslib':'CASUSER(sbxjen)', 'name':'processedImages'},
                         imagefunctions=[{'functionoptions':{'functionType':'RESIZE', 'width':IMG_SIZE, 'height':IMG_SIZE}}],
                         imagetable={'caslib':'CASUSER(sbxjen)', 'name':firstPatientName})

NOTE: Table 00CBA091FA4AD62CC3200A657AEB957E contains compressed images.


ERROR: The table processedImages already exists in the session.
ERROR: The action stopped due to errors.


<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.000351s</span> &#183; <span class="cas-memory">mem 0.123MB</span></small></p>

conn.image.processImages sometimes workw for other image data. Note the bit length overflow
code 17 bits 6->7

In [141]:
IMG_SIZE = 100
conn.image.processImages(casout={'caslib':'CASUSER(sbxjen)', 'name':'otherProcessedImages'},
                         imagefunctions=[{'functionoptions':{'functionType':'RESIZE', 'width':IMG_SIZE, 'height':IMG_SIZE}}],
                         imagetable={'caslib':'CASUSER(sbxjen)', 'name':'train'})

NOTE: Table TRAIN contains compressed images.


ERROR: The table otherProcessedImages already exists in the session.
ERROR: The action stopped due to errors.


<div class="cas-output-area"></div>
<p class="cas-results-performance"><small><span class="cas-elapsed">elapsed 0.000518s</span> &#183; <span class="cas-memory">mem 0.123MB</span></small></p>

CNN: createModel and addLayers

In [ ]:
conn.deepLearn.createModel(
    model={'name':'LeNet', 'replace':True},
    type='CNN'
)